## Neural networks

In [1]:
import os
import numpy as np
from tqdm import tqdm
from random import shuffle

In [2]:
%matplotlib inline
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:

def one_hot_label(img):
    # img is the file name of a configuration
    label, T, _, = img.split('_')
    if label == 'low':
        ohl = np.array([1, 0])
    elif label == 'high':
        ohl = np.array([0, 1])
    return ohl, T

def data_with_label(data_path, *args):
    data = []
    if args:  # The args are the categories in the data path that separate the classes/categories
        for category in args:
            category_path = os.path.join(data_path, category)  # Category folder
            for i in tqdm(os.listdir(category_path)):
                path = os.path.join(category_path, i) 
                img = np.load(path)  # Assuming the files are NumPy arrays
                data.append([img.astype(dtype='float32'), one_hot_label(i)])
        shuffle(data)
    return data



In [4]:
train_path = r'C:\Users\frenc\Documents\dottorato\ising\train'

valid_path = r'C:\Users\frenc\Documents\dottorato\ising\validation'


In [5]:

train_set = data_with_label(train_path, 'low', 'high')

valid_set = data_with_label(valid_path, 'low', 'high')

100%|██████████| 1600/1600 [00:00<00:00, 6242.14it/s]


In [6]:
tr_img_data = np.array([i[0] for i in train_set])
tr_lbl_data = np.array([i[1][0] for i in train_set])
tr_Temp_data = np.array([i[1][1] for i in train_set])
val_img_data = np.array([i[0] for i in valid_set])
val_lbl_data = np.array([i[1][0] for i in valid_set])
val_Temp_data = np.array([float(i[1][1]) for i in valid_set])

In [7]:
# Assuming `tr_img_data` and `tr_lbl_data` are already loaded numpy arrays
train_images = torch.tensor(tr_img_data, dtype=torch.float32)  # Input data (e.g., (N, 20, 20))
train_labels = torch.tensor(tr_lbl_data, dtype=torch.float32)  # Labels (e.g., (N,))
val_images = torch.tensor(val_img_data, dtype=torch.float32)  # Input data (e.g., (N, 20, 20))
val_labels = torch.tensor(val_lbl_data, dtype=torch.float32)  

In [8]:
def calculate_accuracy(predictions, true_labels):
    # Convert predictions and true_labels into numpy arrays if they aren't already
    
    predictions = predictions.detach().cpu().numpy()  # Move to CPU and detach from the graph
    true_labels = true_labels.detach().cpu().numpy() 

    # Calculate the number of correct predictions
    correct_predictions = np.sum(np.argmax(predictions, axis=1) == np.argmax(true_labels, axis=1))
    total_predictions = len(predictions)

    # Return accuracy
    accuracy = correct_predictions / total_predictions
    return accuracy

In [9]:
def train_model(model, train_images, train_labels, val_images, val_labels, epochs=15, batch_size=32, lr=0.001):
    """
    Train a model using the provided training data.

    Parameters:
        model: The model to train (instance of nn.Module).
        train_images: Tensor of input data (e.g., shape (N, 20, 20)).
        train_labels: Tensor of labels (e.g., shape (N,)).
        val_images: Tensor of validation images.
        val_labels: Tensor of validation labels.
        epochs: Number of training epochs.
        batch_size: Batch size for training.
        lr: Learning rate.
    """
    # Define the loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Convert to the correct device (GPU if available)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    train_images, train_labels = train_images.to(device), train_labels.to(device)
    val_images, val_labels = val_images.to(device), val_labels.to(device)
    
    # Training loop
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        epoch_loss = 0 
        accuracy_epoch = []
        
        # Shuffle data and iterate in mini-batches
        perm = torch.randperm(train_images.size(0))
        for i in range(0, len(train_images), batch_size):
            indices = perm[i:i+batch_size]
            batch_images, batch_labels = train_images[indices], train_labels[indices]

            # Forward pass: Compute predicted y by passing x to the model
            outputs = model(batch_images)
            
            accuracy = calculate_accuracy(outputs, batch_labels)
            # Calculate the loss
            loss = criterion(outputs, batch_labels)

            epoch_loss += loss.item()
            # Zero gradients, backward pass, optimizer step
            optimizer.zero_grad()  # Clear previous gradients
            loss.backward()  # Backpropagate the loss
            optimizer.step()  # Update the model parameters
            accuracy_epoch.append(accuracy)
        
        # Calculate average loss and accuracy for the epoch
        epoch_loss = epoch_loss / (len(train_images) // batch_size)
        accuracy_epoch = np.mean(accuracy_epoch)

        # Validation phase
        model.eval()  # Set the model to evaluation mode
        with torch.no_grad():  # No need to compute gradients during validation
            val_outputs = model(val_images)
            val_loss = criterion(val_outputs, val_labels)
            val_accuracy = calculate_accuracy(val_outputs, val_labels)

        # Print loss and accuracy statistics after each epoch
        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy_epoch:.4f}, "
              f"Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}")

In [14]:

class DenseNetwork(nn.Module):
    
    def __init__(self):
        super(DenseNetwork, self).__init__()
        # Flatten layer isn't needed in PyTorch, input shape is handled in the forward pass
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(20 * 20, 100)  # Flattened 20x20 input to 10 output
        self.fc2 = nn.Linear(100, 2)  # 10 to 2 outputs

    def forward(self, x):
        x = self.flatten(x)
        x = torch.sigmoid(self.fc1(x))  # Sigmoid activation
        x = self.fc2(x) 
        x = F.sigmoid(x)
        
        return x  # Apply softmax activation on the output layer

In [15]:
model = DenseNetwork()

# Call the train function
train_model(model,
            train_images,
            train_labels, 
            val_images = val_images, 
            val_labels = val_labels, 
            epochs = 20, 
            batch_size = 32,
            lr = 0.001
            )

Epoch 1/20, Train Loss: 0.6756, Train Accuracy: 0.5631, Val Loss: 0.6202, Val Accuracy: 0.7067
Epoch 2/20, Train Loss: 0.4709, Train Accuracy: 0.8869, Val Loss: 0.3453, Val Accuracy: 0.9220
Epoch 3/20, Train Loss: 0.2664, Train Accuracy: 0.9200, Val Loss: 0.2333, Val Accuracy: 0.9200
Epoch 4/20, Train Loss: 0.2044, Train Accuracy: 0.9292, Val Loss: 0.1999, Val Accuracy: 0.9290
Epoch 5/20, Train Loss: 0.1798, Train Accuracy: 0.9360, Val Loss: 0.1874, Val Accuracy: 0.9337
Epoch 6/20, Train Loss: 0.1642, Train Accuracy: 0.9406, Val Loss: 0.2089, Val Accuracy: 0.9227
Epoch 7/20, Train Loss: 0.1547, Train Accuracy: 0.9417, Val Loss: 0.1833, Val Accuracy: 0.9303
Epoch 8/20, Train Loss: 0.1434, Train Accuracy: 0.9446, Val Loss: 0.1786, Val Accuracy: 0.9327
Epoch 9/20, Train Loss: 0.1325, Train Accuracy: 0.9486, Val Loss: 0.1832, Val Accuracy: 0.9270
Epoch 10/20, Train Loss: 0.1240, Train Accuracy: 0.9523, Val Loss: 0.1766, Val Accuracy: 0.9323
Epoch 11/20, Train Loss: 0.1168, Train Accuracy: 